In [2]:
import cv2
import os
import numpy as np
from tensorflow.keras.utils import to_categorical
from keras.models import Model
from keras.layers import Input, Dense, Activation, GlobalAveragePooling2D, Dropout, Conv2D, BatchNormalization, MaxPooling2D, Flatten
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import confusion_matrix, classification_report
import matplotlib.pyplot as plt
from mlxtend.plotting import plot_confusion_matrix
from tensorflow.keras.applications.densenet import DenseNet169
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.layers import Reshape, Conv2D, BatchNormalization, MaxPooling2D, Flatten, Dense, Input, GlobalAveragePooling2D, GlobalMaxPooling2D

In [3]:
!pip install codecarbon
!pip install pyJoules

# Energy tracking imports
import pynvml
from codecarbon import EmissionsTracker
from pyJoules.device.rapl_device import RaplPackageDomain
from pyJoules.energy_meter import measure_energy

# Initialize pynvml for power usage tracking
pynvml.nvmlInit()

# Define the target shape for resizing images
train_path = '/kaggle/input/realwaste/realwaste-main/RealWaste'
target_shape = (290, 290, 3)

def load_dataset(path, target_shape):
    class_folders = os.listdir(path)
    filenames = []
    labels = []
    class_image_count = {}  # Dictionary to keep track of the number of images per class

    for i, class_folder in enumerate(class_folders):
        folder_path = os.path.join(path, class_folder)
        
        if not os.path.isdir(folder_path):
            continue
        
        class_image_count[class_folder] = 0
        
        for filename in os.listdir(folder_path):
            image_path = os.path.join(folder_path, filename)
            try:
                image = cv2.imread(image_path)
                if image is None:
                    continue
                resized_image = cv2.resize(image, target_shape[:2])
                filenames.append(resized_image)
                labels.append(i)
                class_image_count[class_folder] += 1
            except Exception as e:
                print(f"Error processing image: {image_path} - {e}")

    X = np.array(filenames)
    y = np.array(labels)

    X = X.astype('float32') / 255.0
    y = to_categorical(y)

    for class_name, count in class_image_count.items():
        print(f"Class '{class_name}' has {count} images.")

    return X, y

X, y = load_dataset(train_path, target_shape)

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1, random_state=42)

print("Shape of X_train, y_train:", X_train.shape, y_train.shape)
print("Shape of X_test, y_test:", X_test.shape, y_test.shape)
print("Shape of X_val, y_val:", X_val.shape, y_val.shape)

    

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


Class 'Metal' has 790 images.
Class 'Glass' has 420 images.
Class 'Paper' has 500 images.
Class 'Vegetation' has 436 images.
Class 'Cardboard' has 461 images.
Class 'Textile Trash' has 318 images.
Class 'Food Organics' has 411 images.
Class 'Plastic' has 921 images.
Class 'Miscellaneous Trash' has 495 images.
Shape of X_train, y_train: (3420, 290, 290, 3) (3420, 9)
Shape of X_test, y_test: (951, 290, 290, 3) (951, 9)
Shape of X_val, y_val: (381, 290, 290, 3) (381, 9)


In [4]:
def squeeze_excite_block(input, filters, se_ratio):
    # Squeeze step
    se = GlobalAveragePooling2D()(input)
    se = Dense(filters // se_ratio, activation='relu')(se)
    se = Dense(filters, activation='sigmoid')(se)
    # Excitation step
    x = tf.keras.layers.multiply([input, se])
    return x

In [5]:
import tensorflow as tf
from keras.layers import Input, Dense,UpSampling2D,AveragePooling2D, Activation, Dropout, Conv2D, BatchNormalization, MaxPooling2D, Flatten, concatenate

from keras.layers import concatenate, multiply
from keras.models import Model
from tensorflow.keras.applications.densenet import DenseNet201

def create_parallel_cnn_model(input_shape, num_classes):
    MainInput = Input(shape=input_shape)
    dn169 = DenseNet201(weights='imagenet', include_top=False, input_shape=input_shape, pooling=None)(MainInput)

    def squeeze_excite_block(input, filters, se_ratio):
        se = tf.keras.layers.GlobalAveragePooling2D()(input)  # Global Average Pooling along spatial dimensions
        se = tf.keras.layers.Reshape((1, 1, filters))(se)
        se = tf.keras.layers.Dense(filters // se_ratio, activation='relu')(se)
        se = tf.keras.layers.Dense(filters, activation='sigmoid')(se)
        x = tf.keras.layers.multiply([input, se])
        return x
    # Branch with MaxPooling and Squeeze-Excitation
    conv1 = Conv2D(filters=128, kernel_size=(7, 7), activation="relu", padding="same", kernel_initializer='he_normal')(dn169)
    conv1 = BatchNormalization()(conv1)
    conv1 = Conv2D(filters=128, kernel_size=(5, 5), activation="relu", padding="same", kernel_initializer='he_normal')(conv1)
    conv1 = BatchNormalization()(conv1)
    conv1 = MaxPooling2D(strides=(2, 2), padding="same")(conv1)
    conv1 = Conv2D(filters=256, kernel_size=(7, 7), activation="relu", padding="same", kernel_initializer='he_normal')(conv1)
    conv1 = BatchNormalization()(conv1)
    conv1 = Conv2D(filters=256, kernel_size=(5, 5), activation="relu", padding="same", kernel_initializer='he_normal')(conv1)
    conv1 = BatchNormalization()(conv1)
    conv1 = MaxPooling2D(pool_size=(2, 2), padding="same")(conv1)
    conv1 = squeeze_excite_block(conv1, 256, 16)  # Using 16 as the SE ratio

    # Branch with AveragePooling and Squeeze-Excitation
    conv2 = Conv2D(filters=128, kernel_size=(7, 7), activation="relu", padding="same", kernel_initializer='he_normal')(dn169)
    conv2 = BatchNormalization()(conv2)
    conv2 = Conv2D(filters=128, kernel_size=(5, 5), activation="relu", padding="same", kernel_initializer='he_normal')(conv2)
    conv2 = BatchNormalization()(conv2)
    conv2 = AveragePooling2D(pool_size=(2, 2), padding="same")(conv2)
    conv2 = Conv2D(filters=256, kernel_size=(7, 7), activation="relu", padding="same", kernel_initializer='he_normal')(conv2)
    conv2 = BatchNormalization()(conv2)
    conv2 = Conv2D(filters=256, kernel_size=(5, 5), activation="relu", padding="same", kernel_initializer='he_normal')(conv2)
    conv2 = BatchNormalization()(conv2)
    conv2 = AveragePooling2D(pool_size=(2, 2), padding="same")(conv2)
    conv2 = squeeze_excite_block(conv2, 256, 16)  # Using 16 as the SE ratio

    # Global Average Pooling to reduce spatial dimensions to (1, 1)
    conv1 = tf.keras.layers.GlobalAveragePooling2D()(conv1)
    conv2 = tf.keras.layers.GlobalAveragePooling2D()(conv2)

    # Concatenate both branches
    merged = concatenate([conv1, conv2])
    # Concatenate both branches
    merged = concatenate([conv1, conv2])

    # Dense layers
    dense1 = Dense(1024, activation="relu")(merged)
    dense2 = Dense(512, activation="relu")(dense1)
    output = Dense(num_classes, activation="softmax")(dense2)

    # Create the model
    model = Model(inputs=MainInput, outputs=output)

    return model




# Create the model
num_classes = y_train.shape[1]
model = create_parallel_cnn_model(target_shape, num_classes)
model.summary()

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate=0.001, epsilon=0.1), metrics=['accuracy'])


74836368/74836368 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 290, 290,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ densenet201         │ (None, 9, 9,      │ 18,321,984 │ input_layer[0][0] │
│ (Functional)        │ 1920)             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, 9, 9, 128) │ 12,042,368 │ densenet201[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_4 (Conv2D)   │ (None, 9, 9, 128) │ 12,042,368 │ densenet201[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalization │ (None, 9, 9, 128) │        512 │ conv2d[0][0]      │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 9, 9, 128) │        512 │ conv2d_4[0][0]    │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_1 (Conv2D)   │ (None, 9, 9, 128) │    409,728 │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_5 (Conv2D)   │ (None, 9, 9, 128) │    409,728 │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 9, 9, 128) │        512 │ conv2d_1[0][0]    │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 9, 9, 128) │        512 │ conv2d_5[0][0]    │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d       │ (None, 5, 5, 128) │          0 │ batch_normalizat… │
│ (MaxPooling2D)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ average_pooling2d   │ (None, 5, 5, 128) │          0 │ batch_normalizat… │
│ (AveragePooling2D)  │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_2 (Conv2D)   │ (None, 5, 5, 256) │  1,605,888 │ max_pooling2d[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_6 (Conv2D)   │ (None, 5, 5, 256) │  1,605,888 │ average_pooling2… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 5, 5, 256) │      1,024 │ conv2d_2[0][0]    │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 5, 5, 256) │      1,024 │ conv2d_6[0][0]    │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_3 (Conv2D)   │ (None, 5, 5, 256) │  1,638,656 │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_7 (Conv2D)   │ (None, 5, 5, 256) │  1,638,656 │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 5, 5, 256) │      1,024 │ conv2d_3[0][0]    │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼─────────────────

 Total params: 50,793,065 (193.76 MB)

 Trainable params: 50,560,937 (192.87 MB)

 Non-trainable params: 232,128 (906.75 KB)

In [6]:

from keras.callbacks import ModelCheckpoint, EarlyStopping
import pynvml
import time
from codecarbon import EmissionsTracker  # For emissions tracking

import pynvml
import time
from keras.callbacks import ModelCheckpoint, EarlyStopping
from codecarbon import EmissionsTracker

# Initialize pynvml for GPU power measurement
pynvml.nvmlInit()

# Callbacks for early stopping and checkpointing
checkpoint = ModelCheckpoint('/kaggle/working/model.weights.h5', monitor='val_accuracy', save_best_only=True, save_weights_only=True)
early_stopping = EarlyStopping(monitor='val_loss', mode='min', patience=25, min_delta=0.001)

# Function to calculate total energy consumption
def get_power_usage(handle):
    return pynvml.nvmlDeviceGetPowerUsage(handle) / 1000  # in watts

# Training function with continuous power and energy measurement
def train_model_with_energy_tracking():
    handle = pynvml.nvmlDeviceGetHandleByIndex(0)  # GPU index 0
    
    # Track emissions
    tracker = EmissionsTracker()
    tracker.start()
    
    # Initialize energy consumption variables
    total_energy_consumption = 0  # in joules
    time_interval = 1  # in seconds
    
    start_time = time.time()
    
    for epoch in range(35):  # Assuming 35 epochs
        # Measure power at the beginning of each epoch
        power_usage = get_power_usage(handle)  # in watts
        print(f"Epoch {epoch + 1} - Power consumption: {power_usage} W")
        
        # Train the model for one epoch
        history = model.fit(X_train, y_train,
                            epochs=1,
                            batch_size=10,
                            validation_data=(X_val, y_val),
                            callbacks=[checkpoint, early_stopping])
        
        # Calculate energy consumption for this epoch
        energy_consumed = power_usage * time_interval  # Energy = Power * Time
        total_energy_consumption += energy_consumed  # Accumulate energy consumption
    
    end_time = time.time()
    total_time = end_time - start_time
    print(f"Total training time: {total_time:.2f} seconds")
    
    # Stop emissions tracking and capture emissions data
    emissions = tracker.stop()
    
    # Print results
    print(f"Total energy consumption: {total_energy_consumption:.2f} Joules")
    print(f"Carbon emissions: {emissions} kg CO2e")

# Train the model and measure energy
train_model_with_energy_tracking()

# Clean up pynvml
pynvml.nvmlShutdown()



[codecarbon INFO @ 12:17:08] [setup] RAM Tracking...
[codecarbon INFO @ 12:17:08] [setup] GPU Tracking...
[codecarbon INFO @ 12:17:08] Tracking Nvidia GPU via pynvml
[codecarbon WARNING @ 12:17:08] Failed to retrieve gpu total energy consumption
Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/site-packages/codecarbon/core/gpu.py", line 116, in _get_total_energy_consumption
    return pynvml.nvmlDeviceGetTotalEnergyConsumption(self.handle)
  File "/opt/conda/lib/python3.10/site-packages/pynvml/nvml.py", line 2039, in nvmlDeviceGetTotalEnergyConsumption
    _nvmlCheckReturn(ret)
  File "/opt/conda/lib/python3.10/site-packages/pynvml/nvml.py", line 765, in _nvmlCheckReturn
    raise NVMLError(ret)
pynvml.nvml.NVMLError_NotSupported: Not Supported
[codecarbon INFO @ 12:17:08] [setup] CPU Tracking...
[codecarbon WARNING @ 12:17:08] No CPU tracking mode found. Falling back on CPU constant mode. 
 Linux OS detected: Please ensure RAPL files exist at \sys\class\powercap\in

Epoch 1 - Power consumption: 33.211 W


[codecarbon INFO @ 12:17:27] Energy consumed for RAM : 0.000049 kWh. RAM Power : 11.759090423583986 W
[codecarbon WARNING @ 12:17:27] Failed to retrieve gpu total energy consumption
Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/site-packages/codecarbon/core/gpu.py", line 116, in _get_total_energy_consumption
    return pynvml.nvmlDeviceGetTotalEnergyConsumption(self.handle)
  File "/opt/conda/lib/python3.10/site-packages/pynvml/nvml.py", line 2039, in nvmlDeviceGetTotalEnergyConsumption
    _nvmlCheckReturn(ret)
  File "/opt/conda/lib/python3.10/site-packages/pynvml/nvml.py", line 765, in _nvmlCheckReturn
    raise NVMLError(ret)
pynvml.nvml.NVMLError_NotSupported: Not Supported
[codecarbon INFO @ 12:17:27] Energy consumed for all GPUs : 0.000000 kWh. Total GPU Power : 0.0 W
[codecarbon INFO @ 12:17:27] Energy consumed for all CPUs : 0.000177 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 12:17:27] 0.000226 kWh of electricity used since the beginning.
[codecarb

 38/342 ━━━━━━━━━━━━━━━━━━━━ 1:06 220ms/step - accuracy: 0.1083 - loss: 2.2092

[codecarbon INFO @ 12:22:57] Energy consumed for RAM : 0.001126 kWh. RAM Power : 11.759090423583986 W
[codecarbon WARNING @ 12:22:57] Failed to retrieve gpu total energy consumption
Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/site-packages/codecarbon/core/gpu.py", line 116, in _get_total_energy_consumption
    return pynvml.nvmlDeviceGetTotalEnergyConsumption(self.handle)
  File "/opt/conda/lib/python3.10/site-packages/pynvml/nvml.py", line 2039, in nvmlDeviceGetTotalEnergyConsumption
    _nvmlCheckReturn(ret)
  File "/opt/conda/lib/python3.10/site-packages/pynvml/nvml.py", line 765, in _nvmlCheckReturn
    raise NVMLError(ret)
pynvml.nvml.NVMLError_NotSupported: Not Supported
[codecarbon INFO @ 12:22:57] Energy consumed for all GPUs : 0.000000 kWh. Total GPU Power : 0.0 W
[codecarbon INFO @ 12:22:57] Energy consumed for all CPUs : 0.004072 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 12:22:57] 0.005198 kWh of electricity used since the beginning.


...

[codecarbon INFO @ 13:13:09] Energy consumed for RAM : 0.010954 kWh. RAM Power : 11.759090423583986 W
[codecarbon WARNING @ 13:13:09] Failed to retrieve gpu total energy consumption
Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/site-packages/codecarbon/core/gpu.py", line 116, in _get_total_energy_consumption
    return pynvml.nvmlDeviceGetTotalEnergyConsumption(self.handle)
  File "/opt/conda/lib/python3.10/site-packages/pynvml/nvml.py", line 2039, in nvmlDeviceGetTotalEnergyConsumption
    _nvmlCheckReturn(ret)
  File "/opt/conda/lib/python3.10/site-packages/pynvml/nvml.py", line 765, in _nvmlCheckReturn
    raise NVMLError(ret)
pynvml.nvml.NVMLError_NotSupported: Not Supported
[codecarbon INFO @ 13:13:09] Energy consumed for all GPUs : 0.000000 kWh. Total GPU Power : 0.0 W
[codecarbon INFO @ 13:13:09] Energy consumed for all CPUs : 0.039616 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 13:13:09] 0.050571 kWh of electricity used since the beginning.


342/342 ━━━━━━━━━━━━━━━━━━━━ 78s 227ms/step - accuracy: 0.9997 - loss: 0.0033 - val_accuracy: 0.8845 - val_loss: 0.5164


[codecarbon INFO @ 13:13:21] Energy consumed for RAM : 0.010994 kWh. RAM Power : 11.759090423583986 W
[codecarbon WARNING @ 13:13:21] Failed to retrieve gpu total energy consumption
Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/site-packages/codecarbon/core/gpu.py", line 116, in _get_total_energy_consumption
    return pynvml.nvmlDeviceGetTotalEnergyConsumption(self.handle)
  File "/opt/conda/lib/python3.10/site-packages/pynvml/nvml.py", line 2039, in nvmlDeviceGetTotalEnergyConsumption
    _nvmlCheckReturn(ret)
  File "/opt/conda/lib/python3.10/site-packages/pynvml/nvml.py", line 765, in _nvmlCheckReturn
    raise NVMLError(ret)
pynvml.nvml.NVMLError_NotSupported: Not Supported
[codecarbon INFO @ 13:13:21] Energy consumed for all GPUs : 0.000000 kWh. Total GPU Power : 0.0 W
[codecarbon INFO @ 13:13:21] Energy consumed for all CPUs : 0.039759 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 13:13:21] 0.050752 kWh of electricity used since the beginning.
[codecarb

Total training time: 3368.26 seconds
Total energy consumption: 1538.44 Joules
Carbon emissions: 0.0070418797565474835 kg CO2e


/opt/conda/lib/python3.10/site-packages/codecarbon/output_methods/file.py:52: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, pd.DataFrame.from_records([dict(total.values)])])


In [7]:
# Load the best weights
# Load the best weights
model.load_weights('/kaggle/working/model.weights.h5')


# Plot the training loss and accuracy
N = len(history.history['loss'])
plt.style.use("ggplot")
plt.figure()
plt.plot(np.arange(0, N), history.history["loss"], label="train_loss")
plt.plot(np.arange(0, N), history.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, N), history.history["accuracy"], label="train_acc")
plt.plot(np.arange(0, N), history.history["val_accuracy"], label="val_acc")
plt.title("Training Loss and Accuracy")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="center right")
plt.savefig("parallelCNN_Model")

# Determine number of epochs trained
num_epochs_trained = len(history.history['val_loss'])
print(f"Number of epochs trained: {num_epochs_trained}")

NameError: name 'history' is not defined

In [ ]:
# Evaluate the model on the test set
val_loss, val_accuracy = model.evaluate(X_test, y_test, verbose=0)
print("Test loss:", val_loss)
print("Test accuracy:", val_accuracy)

In [ ]:
# Make predictions on the test set
y_pred = model.predict(X_test)
y_pred = np.argmax(y_pred, axis=1)

# Convert one-hot encoded labels back to categorical labels
y_test_labels = np.argmax(y_test, axis=1)

report = classification_report(y_test_labels, y_pred)
print(report)

In [ ]:
from sklearn.metrics import classification_report

# Define class labels
# Define class labels
class_labels = ['Cardboard', 'Glass', 'Paper', 'Trash', 'Metal', 'Plastic']
# Make predictions on the test set

# Make predictions on the test set
y_pred = model.predict(X_test)
y_pred = np.argmax(y_pred, axis=1)

# Convert one-hot encoded labels back to categorical labels
y_test_labels = np.argmax(y_test, axis=1)

# Generate a classification report with specified target names
report = classification_report(y_test_labels, y_pred, target_names=class_labels)
print(report)


In [ ]:
# Determine number of epochs trained
num_epochs_trained = len(history.history['val_loss'])
print(f"Number of epochs trained: {num_epochs_trained}")

# History plotting
epochs = range(1, num_epochs_trained + 1)
fig, ax = plt.subplots(1, 2)
train_acc = history.history['accuracy']
train_loss = history.history['loss']
val_acc = history.history['val_accuracy']
val_loss = history.history['val_loss']
fig.set_size_inches(15, 5)

# Plotting accuracy
ax[0].plot(epochs, train_acc, 'go-', label='Training Accuracy')
ax[0].plot(epochs, val_acc, 'ro-', label='Validation Accuracy')
ax[0].set_title('Model Accuracy')
ax[0].legend()
ax[0].set_xlabel("Epochs")
ax[0].set_ylabel("Accuracy")

# Plotting loss
ax[1].plot(epochs, train_loss, 'g-o', label='Training Loss')
ax[1].plot(epochs, val_loss, 'r-o', label='Validation Loss')
ax[1].set_title('Model Loss')
ax[1].legend()
ax[1].set_xlabel("Epochs")
ax[1].set_ylabel("Loss")

# Save the figure as a PDF
plt.savefig('training_plot.pdf', format='pdf')

# Show the plot
plt.show()


In [ ]:
import seaborn as sns
# Set a seaborn style without grid lines
sns.set(style="white", rc={"axes.grid": False})

# Determine the number of epochs trained
num_epochs_trained = len(history.history['val_loss'])
print(f"Number of epochs trained: {num_epochs_trained}")

# Define custom colors
training_color = '#001f3f'  # dark blue color
validation_color = '#FF4136'  # dark red color

# Plotting training history
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Plotting accuracy
axes[0].plot(range(1, num_epochs_trained + 1), history.history['accuracy'], 'o-', color=training_color, label='Training Accuracy')
axes[0].plot(range(1, num_epochs_trained + 1), history.history['val_accuracy'], 'o-', color=validation_color, label='Validation Accuracy')
axes[0].set_title('Model Accuracy')
axes[0].legend()
axes[0].set_xlabel("Epochs")
axes[0].set_ylabel("Accuracy")

# Plotting loss
axes[1].plot(range(1, num_epochs_trained + 1), history.history['loss'], 'o-', color=training_color, label='Training Loss')
axes[1].plot(range(1, num_epochs_trained + 1), history.history['val_loss'], 'o-', color=validation_color, label='Validation Loss')
axes[1].set_title('Model Loss')
axes[1].legend()
axes[1].set_xlabel("Epochs")
axes[1].set_ylabel("Loss")

# Beautify the plots
for ax in axes:
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)

# Save the figure as a PDF
plt.savefig('journal_training_plot.pdf', format='pdf', bbox_inches='tight')

# Show the plot
plt.show()


In [ ]:
import seaborn as sns
from sklearn.metrics import confusion_matrix

# ...

# Evaluate the model on the test set
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = np.argmax(y_test, axis=1)

# Get class labels in the desired order
#class_labels = ['Metal', 'White Glass', 'Biological', 'Paper', 'Brown Glass', 'Battery', 'Trash', 'Cardboard', 'Shoes', 'Clothes', 'Plastic', 'Green Glass']
# Define class labels
class_labels = ['Cardboard', 'Glass', 'Paper', 'Trash', 'Metal', 'Plastic']
# Calculate confusion matrix
conf_matrix = confusion_matrix(y_true, y_pred_classes)

# Plot the confusion matrix using seaborn
plt.figure(figsize=(10, 8))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='inferno',xticklabels=class_labels, yticklabels=class_labels)

# Customize the font properties for labels and title
title_font = {'fontname': 'Times New Roman', 'size': '14', 'color': 'black', 'weight': 'normal'}
plt.xlabel('Predicted', **title_font)
plt.ylabel('True', **title_font)
plt.title('Confusion Matrix', **title_font)

# Save the confusion matrix plot as a PDF file
plt.savefig('confusion_matrix.pdf', format='pdf', bbox_inches='tight')

# Show the plot
plt.show()


In [ ]:
from sklearn.metrics import matthews_corrcoef, cohen_kappa_score

# Evaluate the model on the test set
y_pred_classes = np.argmax(y_pred, axis=1)

# Calculate Matthews correlation coefficient
mcc = matthews_corrcoef(y_true, y_pred_classes)
print(f'Matthews Correlation Coefficient (MCC): {mcc:.4f}')

# Calculate Cohen's kappa score
kappa = cohen_kappa_score(y_true, y_pred_classes)
print(f'Cohen\'s Kappa Score: {kappa:.4f}')


In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Calculate overall metrics
overall_accuracy = accuracy_score(y_true, y_pred_classes)
overall_precision = precision_score(y_true, y_pred_classes, average='weighted')
overall_recall = recall_score(y_true, y_pred_classes, average='weighted')
overall_f1_score = f1_score(y_true, y_pred_classes, average='weighted')

# Calculate specificity
overall_specificity = np.sum(np.diag(conf_matrix)) / np.sum(conf_matrix)

# Print overall metrics
print(f'Overall Accuracy: {overall_accuracy:.4f}')
print(f'Overall Precision: {overall_precision:.4f}')
print(f'Overall Recall: {overall_recall:.4f}')
print(f'Overall Specificity: {overall_specificity:.4f}')
print(f'Overall F1 Score: {overall_f1_score:.4f}')


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Calculate overall metrics
overall_accuracy = accuracy_score(y_true, y_pred_classes)
overall_precision = precision_score(y_true, y_pred_classes, average='weighted')
overall_recall = recall_score(y_true, y_pred_classes, average='weighted')
overall_f1_score = f1_score(y_true, y_pred_classes, average='weighted')

# Calculate specificity
overall_specificity = np.sum(np.diag(conf_matrix)) / np.sum(conf_matrix)

# Plotting
metrics_names = ['Accuracy', 'Precision', 'Recall', 'Specificity', 'F1 Score']
metrics_values = [overall_accuracy, overall_precision, overall_recall, overall_specificity, overall_f1_score]

fig, ax = plt.subplots(figsize=(10, 6))
bars = ax.bar(metrics_names, metrics_values, color=['blue', 'green', 'orange', 'purple', 'red'])

# Add values above the bars
for bar, value in zip(bars, metrics_values):
    ax.text(bar.get_x() + bar.get_width() / 2 - 0.15, bar.get_height() + 0.002, f'{value:.2f}', fontsize=10)

# Set plot title and labels
plt.title('Overall Performance Metrics')
plt.ylabel('Metric Value')
plt.ylim([0, 1])  # Adjust the y-axis limits if necessary

# Add x-axis title
plt.xlabel('Metrics')

# Save the plot as a PDF file
plt.savefig('overall_metrics_plot.pdf', format='pdf', bbox_inches='tight')

# Show the plot
plt.show()




In [ ]:
import pynvml

pynvml.nvmlInit()
handle = pynvml.nvmlDeviceGetHandleByIndex(0)  # GPU index 0
power_usage = pynvml.nvmlDeviceGetPowerUsage(handle) / 1000  # Power usage in watts
print(f"Power consumption: {power_usage} W")


In [ ]:
!pip install codecarbon
from codecarbon import EmissionsTracker

tracker = EmissionsTracker()
tracker.start()

# Your training code here

tracker.stop()
